In [1]:
import sys

from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
import torch

sys.path.insert(0, '..')

from scripts.dataset_duplicate_image import DuplicateImageDataset
from scripts.model import SiameseDuplicateImageNetwork

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
model = SiameseDuplicateImageNetwork().to(device)

train_dataset = DuplicateImageDataset('../data/Training Data', transforms=[Resize((224, 224), antialias=True)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, persistent_workers=True)

test_dataset = DuplicateImageDataset('../data/Test Data', transforms=[Resize((224, 224), antialias=True)])
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=8, persistent_workers=True)

print(model)

SiameseDuplicateImageNetwork(
  (model): Sequential(
    (0): SiameseNetwork(
      (net1): Sequential(
        (0): Sequential(
          (0): Sequential(
            (0): ConvNormActivation(
              (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): Hardswish()
            )
            (1): InvertedResidual(
              (block): Sequential(
                (0): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
                  (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                  (2): ReLU(inplace=True)
                )
                (1): ConvNormActivation(
                  (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
                  (1): BatchNorm2d(16, eps=0.001, mom

In [4]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.RMSprop(
    [params for params in model.parameters() if params.requires_grad],
    lr=1e-3,
    alpha=0.9
)

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    model.train_loop(train_dataloader, loss_fn, optimizer, device)
    model.test_loop(test_dataloader, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
loss: 0.711110  [   64/32524]
loss: 0.584471  [ 6464/32524]
